In [42]:
import pathlib
import json
import os
import sqlite3
import sys
import time
from datetime import datetime, timedelta
import urllib.request
import csv
from google_apis.sheets import Sheets_API

In [2]:
# initialize global variables
SHEET_ID = ""
PREV_PLAYED = ""
AIMLAB_DB_PATH = os.path.abspath(os.path.join(os.getenv("APPDATA"), os.pardir, "LocalLow\\statespace\\aimlab_tb\\klutch.bytes"))

In [62]:
def parse_config():
    try:
        with open("./config.json", "r") as json_file:
            json_data = json.load(json_file)
        
        last_played = datetime.strptime(json_data["scenario_data"]["last_played"], "%Y-%m-%d %H:%M:%S")
        SHEET_ID = json_data["sheets"]["id"]
        cs_level_ids = None

        # Match scenario difficulty
        difficulty = json_data["scenario_data"]["playlist_type"].lower()
        match difficulty:
            case "beginner":
                cs_level_ids = json_data["scenario_data"]["beginner_scenarios"]
            case "intermediate":
                cs_level_ids = json_data["scenario_data"]["intermediate_scenarios"]
            case _:
                print("Invalid playlist type")
                raise ValueError
    except:
        print("Invalid json file")
        raise ValueError
    return (json_data, last_played, SHEET_ID, cs_level_ids)

config_data, PREV_PLAYED, SHEET_ID, cs_level_ids = parse_config()

In [4]:
# Create Google API to do stuff with
def initalize_apis(spread_id):
    SCOPES = [
        "https://www.googleapis.com/auth/spreadsheets",
    ]

    # When there is pre-determined credentials/token path already
    sheet = Sheets_API(scopes = SCOPES, ID = spread_id)
    return sheet

sheet = initalize_apis(spread_id = SHEET_ID)


In [22]:
def update_config(json_data):
    with open("config.json", "w") as jsonFile:
        json.dump(config_data, jsonFile, indent=4, sort_keys=True)

testing data to see if the row updates

In [30]:
# starting from whatever the earliest row is - update through
test_data = [12, 53, 14]
sheet.update_scenario(test_data, 0)

AttributeError: 'Sheets_API' object has no attribute 'update_scenario'

In [24]:
# Open db connection
con = sqlite3.connect(AIMLAB_DB_PATH)
cur = con.cursor()

# Get scores from the database
for scenario_index, item in enumerate(cs_level_ids.items()):
    csid, name = item
    
    # Query for new data to add
    cur.execute(
        f"SELECT score, endedAt FROM TaskData WHERE taskName = ? AND endedAt > date(?) ORDER BY endedAt",
        [csid, PREV_PLAYED])
    result = cur.fetchall()

    # Update the last played within config file
    config_data["scenario_data"]["last_played"] = result[-1][1]
    update_config(config_data)

    update_data = parse_query(result)
    
    # Update Sheet
    sheet.update_scenario(values, scenario_index, len(values))


In [ ]:
def parse_query(result)
    error_offset, parsed_values = 0, []

    # iterate through in chunks of 3
    for i in range(len(result)):

        # Every 3 indicies create an interval for the day
        if (i % 3) == 2 - error_offset:

            # Check if dates of beginning and end of interval have same date
            if result[i - 2][1][:10] != result[i][1][:10]:
                errors = 1
                # Check if middle scenario was also corrupted
                if result[i - 2][1][:10] != result[i - 1][1][:10]:
                    errors += 1
                
                # Update error offset
                error_offset += errors
                
            # Compute the intervals for the day
            row_data = result[i - 2:i + 1 - errors]
            scores = [x[0] for x in row_data]
            parsed_values.append(scores)
        
    return parsed_values


In [16]:
da = "2024-01-22 18:33:42"
a = datetime.strptime(da, "%Y-%m-%d %H:%M:%S")
a

datetime.datetime(2024, 1, 22, 18, 33, 42)

In [36]:
cur.execute(
    f"SELECT score, endedAt FROM TaskData WHERE taskName = ? AND endedAt > date(?) ORDER BY endedAt",
    ["CsLevel.Lowgravity56.VT Dynam.RQCD1Z", PREV_PLAYED])
error_res = cur.fetchall()
error_res

[(495, '2024-01-13 00:59:08'),
 (551, '2024-01-13 01:00:19'),
 (512, '2024-01-13 01:01:27'),
 (639, '2024-01-14 03:09:21'),
 (607, '2024-01-14 03:10:27'),
 (588, '2024-01-14 03:11:38'),
 (553, '2024-01-15 02:56:15'),
 (691, '2024-01-15 02:57:32'),
 (659, '2024-01-15 02:58:38'),
 (620, '2024-01-16 13:09:16'),
 (617, '2024-01-16 13:10:26'),
 (665, '2024-01-16 13:11:37'),
 (585, '2024-01-17 13:13:58'),
 (636, '2024-01-17 13:15:06'),
 (669, '2024-01-17 13:16:13'),
 (607, '2024-01-18 12:12:00'),
 (626, '2024-01-18 12:13:06'),
 (691, '2024-01-19 13:14:13'),
 (662, '2024-01-19 13:15:19'),
 (708, '2024-01-19 13:16:25'),
 (588, '2024-01-20 15:19:23'),
 (604, '2024-01-20 15:20:32'),
 (525, '2024-01-20 15:21:38'),
 (591, '2024-01-21 13:33:56'),
 (669, '2024-01-21 13:35:04'),
 (646, '2024-01-21 13:36:10'),
 (601, '2024-01-22 18:28:09'),
 (653, '2024-01-22 18:29:15'),
 (611, '2024-01-22 18:30:21')]

In [48]:
# error correction on the result
values, temp = [], []
error_offset = 0

# iterate through in chunks of 3
for i in range(len(error_res)):

    # every 3 check if a scenario wasn't accounted for
    if (i % 3) == 2 - error_offset:

        errors = 0
        # Check if dates of beginning and end of interval have same date
        # print(error_res[i - 2][1][:10], error_res[i][1][:10])
        if error_res[i - 2][1][:10] != error_res[i][1][:10]:
            errors += 1
            # Check if middle scenario was also corrupted
            if error_res[i - 2][1][:10] != error_res[i - 1][1][:10]:
                errors += 1
            
            # Update error offset
            error_offset += errors
            
        # Add as many 0s as there are errors to current batch
        # Compute the intervals for the day
        row_data = result[i - 2:i + 1 - errors]
        scores = [x[0] for x in row_data]
        values.append(scores)
    #     # sheet.update_scenario(scores, index)
    # print(scores, scenario_index, len(values))
    #     Add current day to values and reset day counter
    #     values.append(temp)


[495, 551, 512]
[639, 607, 588]
[553, 691, 659]
[620, 617, 665]
[585, 636, 669]
[607, 626]
[691, 662, 708]
[588, 604, 525]
[591, 669, 646]
[601, 653, 611]


In [37]:
file_path = 'data.csv'
cs_level_ids = dict()
blacklist = dict()

# Open and read the CSV file
with open(file_path, newline='', encoding='utf-8') as csvfile:
    # Create a CSV reader
    csv_reader = csv.reader(csvfile)

    # Skip the header row
    next(csv_reader)

    # Iterate through the lines
    for splits in csv_reader:
        name = splits[0].replace('"', '')
        cs_level_id = splits[1].replace('"', '')
        cs_level_ids[cs_level_id] = name
        date = datetime.strptime(splits[2].replace('"', '').replace('\n', ''), "%d.%m.%Y").date()
        blacklist[name.lower()] = date

cs_level_ids

{'CsLevel.VT Empyrean.VT Angle.RB668Z': 'VT Angleshot Novice',
 'CsLevel.Lowgravity56.VT x WHJ.RWEZ9J': 'VT x WHJ Smooth Strafe Sphere',
 'CsLevel.Lowgravity56.VT Adjus.ROJETY': 'VT Adjust Track VALORANT',
 'CsLevel.Lowgravity56.VT 3T Wi.RTA5MX': 'VT 3T Wide',
 'CsLevel.Lowgravity56.VT Berry.RUPUHP': 'VT berryTS Static Small',
 'CsLevel.Lowgravity56.VT x WHJ.RWFB5F': 'VT x WHJ 5 Sphere Hipfire Small',
 'CsLevel.VT Empyrean.VT 1w2ts.R21FUT': 'VT 1w2ts Smallflicks VALORANT',
 'CsLevel.Lowgravity56.VT Dynam.RQCD1Z': 'VT Dynamic Reflex Micro',
 'CsLevel.Lowgravity56.VT x WHJ.RWFQU4': 'VT x WHJ Speedswitch Click'}